# IBM Cloud Object Storage (ICOS) をpythonから操作

この説明はQiita [IBM Cloud Object Storage (ICOS) をpythonから操作](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df/)にあります。

参考：
- https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-python&locale=en
- https://ibm.github.io/ibm-cos-sdk-python/reference/services/s3.html

# 1. 資格情報のセット
参考: [1.準備](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#1%E6%BA%96%E5%82%99)

In [ ]:
credencials={
  "apikey": "XXXXXXXXXXXXXXXXXXXXXXXXX-",
  "cos_hmac_keys": {
    "access_key_id": "XXXXXXXXXXXXXXXXXXXXXXXXX",
    "secret_access_key": "XXXXXXXXXXXXXXXXXXXXXXXXX12345678"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key XXXXXXXXXXXXXXXXXXXXXXXXX12345678",
  "iam_apikey_name": "hdm-db2",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::XXXXXXXXXXXXXXXXXXXXXXXXX12345678",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:XXXXXXXXXXXXXXXXXXXXXXXXX12345678"
}

# 2. ibm_boto3　ライブラリの導入
参考: [2. ibm_boto3　ライブラリの導入](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#2-ibm_boto3%E3%83%A9%E3%82%A4%E3%83%96%E3%83%A9%E3%83%AA%E3%81%AE%E5%B0%8E%E5%85%A5)

In [ ]:
!pip install ibm-cos-sdk

# 3. resourceの作成
参考: [3. resourceの作成](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#3-resource%E3%81%AE%E4%BD%9C%E6%88%90)

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
# 自分のエンドポイントに変更する
COS_ENDPOINT = "https://s3.private.jp-tok.cloud-object-storage.appdomain.cloud"

# APIKEYを入れる 例”W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_API_KEY_ID = credencials["apikey"] 

# resource_instance_idを入れる 例"crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_INSTANCE_CRN =  credencials["resource_instance_id"]  

# Create resource
resource = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

# 4. ファイルのアップロード
参考: [4. ファイルのアップロード](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#4-%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E3%82%A2%E3%83%83%E3%83%97%E3%83%AD%E3%83%BC%E3%83%89)

In [ ]:
#とりあえテスト用ファイルダウンロード
import urllib
url = "https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv"
save_file = "newly_confirmed_cases_daily.csv"
data = urllib.request.urlopen(url).read()
with open(save_file, mode="wb") as f:
    f.write(data)

In [ ]:
#アップロード先Bucket名
target_bucket = "python-test-2022" 
#アップロード 元ファイル名
src_file = "newly_confirmed_cases_daily.csv"
#アップロード 先ファイル名(必要ならpathをつける)
target_file ="newly_confirmed_cases_daily.csv"

bucket = resource.Bucket(target_bucket)
bucket.upload_file(src_file, target_file)

In [ ]:
#確認
object = resource.Object(target_bucket, target_file)
response = object.get()
body = response['Body'].read(1000)
print(body.decode('utf-8', "ignore"))

# 5. ファイルのコピー
参考: [5. ファイルのコピー](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#5-%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC)

In [ ]:
# コピー元ファイル情報
copy_source = {
    #  コピー元Bucket名
    "Bucket": "python-test-2022" ,
     #  コピー元File名
    "Key": "newly_confirmed_cases_daily.csv"
}
#コピー先 Bucket名
target_bucket = "python-test-2022" 
#コピー先 ファイル名(必要ならpathをつける)
target_file ="copytest/newly_confirmed_cases_daily.csv"

bucket = resource.Bucket(target_bucket)
bucket.copy(copy_source, target_file)

In [ ]:
#確認
bucket = resource.Bucket(target_bucket)
for object in bucket.objects.all():
    print(object)

# 6. ファイルの削除
参考: [6. ファイルの削除](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#6-%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E5%89%8A%E9%99%A4)

In [ ]:
#削除するファイルが入っている Bucket名
target_bucket = "python-test-2022" 
#削除ファイル名(必要ならpathをつける)
delete_file ="newly_confirmed_cases_daily.csv"

object = resource.Object(target_bucket, delete_file)
object.delete()

In [ ]:
#確認
bucket = resource.Bucket(target_bucket)
for object in bucket.objects.all():
    print(object)

# 7. ファイルのダウンロード
参考: [7. ファイルのダウンロード](https://qiita.com/nishikyon/items/5cf4eaaea380bb0db1df#7-%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89)

In [ ]:
#ダウンロードするファイルが入っている Bucket名
source_bucket = "python-test-2022" 
#ダウンロードするファイル名(必要ならpathをつける)
source_file ="copytest/newly_confirmed_cases_daily.csv"
# ローカルの保存先ファイル名(必要ならpathをつける)
local_file ="icos_newly_confirmed_cases_daily.csv"

object = resource.Object(source_bucket, source_file)
object.download_file(local_file)

In [ ]:
import os

#ダウンロード確認
if(os.path.isfile(local_file) == True):
    print('Download OK')
else:
    print('Download Failed')